# SimHash
Using hashing for Kaggl's Question Duplicate Pair Detection. Content from the blog post [SimHash for question deduplication](http://datawhatnow.com/simhash-question-deduplicatoin/).

## Libraries

In [1]:
# Utility libraries
import pandas as pd

# Dataset
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams
from simhash import Simhash
from sklearn.model_selection import train_test_split
from multiprocessing import Pool # We use pool to speed up feature creation

#Metrics
from sklearn.metrics import f1_score, log_loss, accuracy_score, precision_score, recall_score

# Models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load the dataset
The dataset (train.csv) comes from the Quao challange on Kaggle. To get the dataset download it from the official competition page on Kaggle https://www.kaggle.com/c/quora-question-pairs/data.

In [2]:
train = pd.read_csv('train.csv', dtype={'question1': str, 'question2': str})

train.question1 = train.question1.astype(str)
train.question2 = train.question2.astype(str)

## Data preprocessing
SimHash is a hashing function that takes chuncks of text as inputs. We have to process the text into chunks first (words, char n-grams, word n-grams).

In [3]:
def tokenize(sequence):
    words = word_tokenize(sequence)
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    return filtered_words

def clean_sequence(sequence):
    tokens = tokenize(sequence)
    return ' '.join(tokens)

def get_word_ngrams(sequence, n=3):
    tokens = tokenize(sequence)
    return [' '.join(ngram) for ngram in ngrams(tokens, n)]

def get_character_ngrams(sequence, n=3):
    sequence = clean_sequence(sequence)
    return [sequence[i:i+n] for i in range(len(sequence)-n+1)]

In [4]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
sequence = "Mary had a little lamb, and she really liked it."
print(clean_sequence(sequence))

Mary little lamb , really liked .


In [6]:
sequence = "Mary had a little lamb, and she really liked it."

print('Tokens:', tokenize(sequence)) 
# ['Mary', 'little', 'lamb', ',', 'really', 'liked', '.']

print('\nWord ngrams:', get_word_ngrams(sequence, 2))  
# ['Mary little', 'little lamb', 'lamb ,', ', really', 'really liked', 'liked .']

print('\nCharacter ngrams:', get_character_ngrams(sequence, 2))  
# ['Ma', 'ar', 'ry', 'y ', ' l', 'li', 'it', 'tt', 'tl', 'le', 'e ', ' l', 'la', 'am', 'mb', 'b ', ' ,', ', ', ' r', 're', 'ea', 'al', 'll', 'ly', 'y ', ' l', 'li', 'ik', 'ke', 'ed', 'd ', ' .']

Tokens: ['Mary', 'little', 'lamb', ',', 'really', 'liked', '.']

Word ngrams: ['Mary little', 'little lamb', 'lamb ,', ', really', 'really liked', 'liked .']

Character ngrams: ['Ma', 'ar', 'ry', 'y ', ' l', 'li', 'it', 'tt', 'tl', 'le', 'e ', ' l', 'la', 'am', 'mb', 'b ', ' ,', ', ', ' r', 're', 'ea', 'al', 'll', 'ly', 'y ', ' l', 'li', 'ik', 'ke', 'ed', 'd ', ' .']


## Simhash examples

In [7]:
q1 = "How can I be a good geologist?"
q2 = "What should I do to be a great geologist?"

print('Tokenize simhash:', Simhash(tokenize(q1)).distance(Simhash(tokenize(q2))))
print('Word ngram simhash:', Simhash(get_word_ngrams(q1)).distance(Simhash(get_word_ngrams(q2))))
print('Tokenize simhash:', Simhash(get_character_ngrams(q1)).distance(Simhash(get_character_ngrams(q2))))

Tokenize simhash: 20
Word ngram simhash: 34
Tokenize simhash: 16


## Feature creation
The code below is kinda messy. I wrote it as a single function just to be able to use the Pool.map() method, otherwise this step takes a lot of time.

In [8]:
def caluclate_simhash_distance(sequence1, sequence2):
    return Simhash(sequence1).distance(Simhash(sequence2))

def get_word_distance(questions):
    q1, q2 = questions.split('_split_tag_')
    q1, q2 = tokenize(q1), tokenize(q2)
    return caluclate_simhash_distance(q1, q2)

def get_word_2gram_distance(questions):
    q1, q2 = questions.split('_split_tag_')
    q1, q2 = get_word_ngrams(q1, 2), get_word_ngrams(q2, 2)
    return caluclate_simhash_distance(q1, q2)

def get_char_2gram_distance(questions):
    q1, q2 = questions.split('_split_tag_')
    q1, q2 = get_character_ngrams(q1, 2), get_character_ngrams(q2, 2)
    return caluclate_simhash_distance(q1, q2)

def get_word_3gram_distance(questions):
    q1, q2 = questions.split('_split_tag_')
    q1, q2 = get_word_ngrams(q1, 3), get_word_ngrams(q2, 3)
    return caluclate_simhash_distance(q1, q2)

def get_char_3gram_distance(questions):
    q1, q2 = questions.split('_split_tag_')
    q1, q2 = get_character_ngrams(q1, 3), get_character_ngrams(q2, 3)
    return caluclate_simhash_distance(q1, q2)

train['questions'] = train['question1'] + '_split_tag_' + train['question2']

In [9]:
train.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,questions
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,What is the story of Kohinoor (Koh-i-Noor) Dia...


In [10]:
# Instead of 8, swap the number with the number of cpu cores/threads you have
pool = Pool(processes=8)

train['tokenize_distance'] = pool.map(get_word_distance, train['questions'])

train['word_2gram_distance'] = pool.map(get_word_2gram_distance, train['questions'])
train['char_2gram_distance'] = pool.map(get_char_2gram_distance, train['questions'])

train['word_3gram_distance'] = pool.map(get_word_3gram_distance, train['questions'])
train['char_3gram_distance'] = pool.map(get_char_3gram_distance, train['questions'])

In [11]:
train.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,questions,tokenize_distance,word_2gram_distance,char_2gram_distance,word_3gram_distance,char_3gram_distance
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,12,1,13,4,17
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,25,15,29,13,25


## Model performance

In [12]:
features = ['tokenize_distance', 'word_2gram_distance', 'char_2gram_distance',
           'word_3gram_distance', 'char_3gram_distance']
target = 'is_duplicate'

X = train[features]
y = train[target]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [15]:
prediction_probas = model.predict_proba(X_test)
predictions = model.predict(X_test)

print('Acc:', accuracy_score(y_test, predictions))
print('LogLoss:', log_loss(y_test, prediction_probas))

Acc: 0.680959212447
LogLoss: 0.55707137604
